In [2]:
import os
import librosa
import numpy as np
import pickle
import hashlib
from scipy.ndimage import maximum_filter, binary_erosion, generate_binary_structure

In [4]:
# Parameters
FAN_VALUE = 15
MAX_TIME_DELTA = 200
DB_FILE = 'fingerprints.pkl'

def fingerprint(audio, sr):
    S = np.abs(librosa.stft(audio))
    S_db = librosa.amplitude_to_db(S, ref=np.max)

    struct = generate_binary_structure(2, 1)
    local_max = maximum_filter(S_db, footprint=struct) == S_db
    background = (S_db == 0)
    detected_peaks = local_max ^ binary_erosion(background)

    peaks = np.argwhere(detected_peaks)
    hashes = []

    for i in range(len(peaks)):
        for j in range(1, FAN_VALUE):
            if i + j < len(peaks):
                freq1, time1 = peaks[i]
                freq2, time2 = peaks[i + j]
                delta_t = time2 - time1
                if 0 < delta_t <= MAX_TIME_DELTA:
                    hash_input = f"{freq1}|{freq2}|{delta_t}"
                    hash_val = hashlib.sha1(hash_input.encode()).hexdigest()[:20]
                    hashes.append((hash_val, time1))
    return hashes

def build_database(folder_path, db_file=DB_FILE):
    database = {}
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.mp3', '.wav', '.flac')):
            path = os.path.join(folder_path, fname)
            print(f"Processing: {fname}")
            audio, sr = librosa.load(path, sr=None, mono=True)
            hashes = fingerprint(audio, sr)
            database[fname] = hashes

    with open(db_file, 'wb') as f:
        pickle.dump(database, f)
    return database

def load_database(db_file=DB_FILE):
    if os.path.exists(db_file):
        with open(db_file, 'rb') as f:
            return pickle.load(f)
    return {}

def match_sample(sample_hashes, database):
    match_counts = {}
    for song, hashes in database.items():
        hash_set = set(h[0] for h in hashes)
        match_count = sum(1 for h, _ in sample_hashes if h in hash_set)
        match_counts[song] = match_count

    if match_counts:
        best_match = max(match_counts, key=match_counts.get)
        return best_match, match_counts[best_match]
    return None, 0


In [5]:
build_database("wav_files", "fingerprints.pkl")

Processing: 05. Loveland, Island.wav
Processing: 06. Love Talkin' (honey it's you).wav


: 